In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
import random

#metti final data >= 2016

data = pd.read_json("finalTrumpPreprocessed.json")
print(len(data))
data = data[data["year"] >= 2016] 
print(len(data))
data = data[data["is_retweet"] == False] 
print(len(data))

#remove links
data["modded1"] = data["text"].str.replace("http\S+"," ")

data["modded1"] = data["modded1"].str.replace("\n", " ")

#remove punctuation
data['modded1'] = data["modded1"].str.replace("[^\w\s]"," ")
data["modded1"] = data["modded1"].str.lower()

sentences = data["modded1"].tolist()
list_train_set = sentences

fullText = " ".join(sentences)
text_in_words = [w for w in fullText.split(' ') if w.strip() != '' or w == '\n']
text_in_words = list(map(lambda s: s.strip(), text_in_words))

print('Corpus length in words:', len(text_in_words))

from collections import Counter
word_freq = dict(Counter(fullText.split()))

not_freq = [x for x in word_freq if word_freq[x] < 19] 

shorter = []
for sentence in sentences:
    t = sentence.split()
    q = sentence
    for el in t:
        if el in not_freq:
            q = q.replace(el," ")
    if q != []:
        shorter.append([q])
        
unique = len(word_freq.keys())-len(not_freq)
print("words", len(word_freq.keys()))
print("not frequent", len(not_freq))
print("remaining", len(word_freq.keys())-len(not_freq))


Using TensorFlow backend.


36159
10325
10325
Corpus length in words: 255726
words 12742
not frequent 11245
remaining 1497


In [3]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

vocab = Counter(fullText.split())
#print(vocab)

corpus = [x.split() for x in sentences]
#corpus

In [4]:
from gensim.models import Word2Vec


w2vec_size = 200
w2 = Word2Vec(corpus, size=w2vec_size, window=5, min_count=1,sg=1)
word_vectors = w2.wv
#print("Number of word vectors: {}".format(len(word_vectors.vocab)))
w2.wv.most_similar(positive=['clinton'])

MAX_NB_WORDS = len(word_vectors.vocab)
MAX_SEQUENCE_LENGTH = 100

from keras.preprocessing.sequence import pad_sequences
import keras.utils as ku


word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(MAX_NB_WORDS))}
#print(word_index)
sequences = [[word_index[t] for t in sent.split()] for sent in sentences[:len(list_train_set)]]
test_sequences = [[word_index[t]  for t in sent.split()] for sent in sentences[len(list_train_set):]]


In [5]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1] #i+1
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]


[[9, 10],
 [9, 10, 15],
 [9, 10, 15, 11],
 [9, 10, 15, 11, 168],
 [9, 10, 15, 11, 168, 390],
 [9, 10, 15, 11, 168, 390, 19],
 [9, 10, 15, 11, 168, 390, 19, 596],
 [9, 10, 15, 11, 168, 390, 19, 596, 4],
 [9, 10, 15, 11, 168, 390, 19, 596, 4, 37],
 [9, 10, 15, 11, 168, 390, 19, 596, 4, 37, 424]]

In [6]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = 4
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    #riprova il padding con pre
    predictors, lab = input_sequences[:,:-1],input_sequences[:,-1] #tolto -1
    label = ku.to_categorical(lab, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)
input_len = max_sequence_len-1

#data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="pre", truncating="post")
#data
print(predictors.shape)
label.shape

(245404, 3)


(245404, 12743)

In [7]:
from keras.layers import Dense, Input, Embedding, Dropout,LSTM,Bidirectional
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

In [8]:
#WEIGHT MATRIX
WV_DIM = w2vec_size

nb_words = total_words
# we initialize the matrix with random numbers
wv_matrix = (np.random.rand(total_words, WV_DIM) - 0.5) / 5.0
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        # words not found in embedding index will be all-zeros.
        wv_matrix[i] = embedding_vector
    except:
        pass
print(nb_words)

12743


In [ ]:
model = Sequential()

model.add(Embedding(nb_words, w2vec_size, mask_zero=False,weights=[wv_matrix],input_length=input_len,trainable=True)) #mask zero = False
model.add(Bidirectional(LSTM(800)))
model.add(Dropout(0.4))
model.add(Dense(nb_words, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

#con 800 meno di 20 epoch! un po' overfitta, 10

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="postelectionGenerator-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
#history = model.fit(predictors, label, validation_split=0.35, epochs=2, verbose=1)#callbacks=callbacks_list)


In [9]:
import json
from keras.models import model_from_json

with open('definitiveMaybe.json', 'r') as json_file:
    model = model_from_json(json_file.read())
model.load_weights("definitiveMaybe13Iters.h5")

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 3, 200)            2548600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1600)              6406400   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 12743)             20401543  
Total params: 29,356,543
Trainable params: 29,356,543
Non-trainable params: 0
_________________________________________________________________


In [10]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    preds = []
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0] 
        token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='post') #max sequence len - 1
        predicted = model.predict_classes(token_list, verbose=0)
        preds.append(predicted)
        output_word = ""
        
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
#    print(preds)
    return seed_text.title()


In [11]:
generate_text("best socks ever", 32, model, 3)

'Best Socks Ever By The Fake News Media Is Working Overtime Just Reported That Ted Cruz Is Falling In The Polls History All Business Is Just At The Beginning Of The End We Will Probably'

In [52]:
rand = sentences
sampling = random.choices(rand, k=10)

seeds = []
for el in sampling:
    i = random.randint(2,5)
    seeds.append(" ".join(el.split()[:i]))
    
for el in seeds:
    print("seed: \" " + el + "\"")
    t = generate_text(el, 30,model, input_len)
    print(t + "\n")

seed: " laurencristmann sharp_trident"
Laurencristmann Sharp_Trident It It Is So Important To Audit The Federal Reserve And Yet Ted Cruz Missed The Vote On The Bill Clinton Is The Only Person That Can Stop Her In

seed: " today i am greatly"
Today I Am Greatly Honored To Receive The Endorsement Of Me By The 16 500 Border Patrol Agents Was The First Time Ever Make America Great Again Trump2016 Votetrumpsc Makeamericagreatagain Video Trump2016 Newyorkvalues In

seed: " makeamericagreatagain"
Makeamericagreatagain Is The Only Person That Can Stop Her In The Debate Polls Except For Cnn Which I Have The Absolute Right To Pardon Myself But Why Would I Call China

seed: " remember michael"
Remember Michael The Republican Party Is Racking Up Record Amounts Of Small Dollar Donations Fueled By Trump Supporters Nypost Thank You To The Amazing Law Enforcement Officers Americafirst Trump2016 Makeamericagreatagain Omaha Nebraska

seed: " happyindependenceday usa"
Happyindependenceday Usa The U S A G

In [56]:
for sentence in sentences:
    s = sentence.split()
    if "talk"  in s and "no" in s and "action" in s and "single" in s:
        print(sentence + "\n")

In [20]:
goodSeeds = ["why is that","we are with","donald trump is","hillary clinton is a bad","great success today at","i am hearing so","get out and vote","melania and i offer","we must stop being"]
    
for el in goodSeeds:
    print("seed: \" " + el + "\"")
    t = generate_text(el, 27,model, 3)
    print(t + "\n")

seed: " why is that"
Why Is That Hillary Clintons Family And Dems Dealings With Russia Are Not Looked At But My Non Dealings Are Made Up By Women Many Already Proven False And Pushed

seed: " we are with"
We Are With You And Your Campaign In The History Of Politics B C I Stand 100 Behind Everything We Do What We Do Best We Pull Together We Join

seed: " donald trump is"
Donald Trump Is A Vote To Cut Taxes Big League For The Middle Class And Business Jobs The Reason Obstruction And Delay At This Rate It Would Take 9 Years

seed: " hillary clinton is a bad"
Hillary Clinton Is A Bad Joke And A Hoax Designed To Target Trump Tom Fitton Judicialwatch A Disgrace To Our Country And Our Workers Our Steel Industry Is In Bad Shape If

seed: " great success today at"
Great Success Today At The Whitehouse Congratulations To The Republic Of Korea On Hosting The Olympics What A Wonderful Opportunity To Show Everyone That You Are Totally Anti Trump Little Jeff

seed: " i am hearing so"
I Am Hearing So

In [29]:
for sentence in sentences:
    s = sentence.split()
    if "why"  in s and "that" in s and "clintons" and "family" in s:
        print(sentence + "\n")

why is that hillary clintons family and dems dealings with russia are not looked at  but my non dealings are 



In [ ]:
finalSeeds = ["donald trump is","hillary clinton is a bad","great success today at"]